In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [ ]:
# 初始化构造数据
from bailian_nlp.preprocessing.build_data import build_pos_fake_data, build_pos_train_and_valid_data
# build_pos_fake_data()
build_pos_train_and_valid_data()



In [ ]:
# 正常训练

from bailian_nlp.modules import BertData

data = BertData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    max_seq_len=128,
    batch_size=128
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    enc_hidden_dim=256
)


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


In [7]:
import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 2
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz from cache at /home/liuxiang/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/liuxiang/.pytorch_pretrained_bert/42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f to temp dir /tmp/tmpstd_2xia
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_

INFO:root:
epoch 1, average train epoch loss=115.24



INFO:root:on epoch 0 by max_f1: 0.868
INFO:root:Saving new best model...


              precision    recall  f1-score   support

       <pad>      0.000     0.000     0.000         0
       [CLS]      1.000     1.000     1.000       352
         B_w      0.887     0.937     0.911      2002
         E_w      0.974     0.984     0.979      1988
         S_n      0.892     0.880     0.886      2426
         B_t      0.842     0.849     0.846       862
         I_t      0.853     0.962     0.904      2193
         E_t      0.915     0.910     0.913       820
         B_v      0.863     0.860     0.862      1698
         I_v      0.773     0.717     0.744       583
         E_v      0.896     0.900     0.898      1684
         B_p      0.813     0.833     0.823       251
         E_p      0.888     0.916     0.902       251
         B_n      0.840     0.834     0.837      2452
         E_n      0.876     0.855     0.865      2399
         I_n      0.845     0.830     0.837      1540
         B_u      0.890     0.915     0.903       673
         E_u      0.963    

INFO:root:
epoch 2, average train epoch loss=39.963



INFO:root:on epoch 1 by max_f1: 0.878
INFO:root:Saving new best model...


              precision    recall  f1-score   support

       <pad>      0.000     0.000     0.000         0
       [CLS]      1.000     1.000     1.000       352
         B_w      0.906     0.932     0.919      2002
         E_w      0.974     0.985     0.980      1988
         S_n      0.895     0.884     0.890      2426
         B_t      0.858     0.861     0.859       862
         I_t      0.879     0.952     0.914      2193
         E_t      0.925     0.913     0.919       820
         B_v      0.874     0.872     0.873      1698
         I_v      0.799     0.741     0.769       583
         E_v      0.911     0.904     0.908      1684
         B_p      0.831     0.845     0.838       251
         E_p      0.895     0.916     0.906       251
         B_n      0.854     0.841     0.848      2452
         E_n      0.878     0.861     0.869      2399
         I_n      0.841     0.850     0.845      1540
         B_u      0.899     0.929     0.914       673
         E_u      0.965    

In [ ]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


In [ ]:
learner.fit(num_epochs, target_metric='f1')

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [8]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


INFO:bailian_nlp.released.pos:load default user_dict in /home/liuxiang/Projects/pos-bert/bailian_nlp/datadrive/dict/user_dict.txt
INFO:bailian_nlp.released.dictionary:本次加载词条数：3
INFO:bailian_nlp.released.dictionary:当前总词条数: 3
ERROR:pytorch_pretrained_bert.modeling:Model name 'bert-base-chinese' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese). We assumed 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz' was a path or url but couldn't find any file associated to this path or url.


AttributeError: 'NoneType' object has no attribute 'to'

In [ ]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
text = '董事'
# text = '一言九鼎'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res